### HOW TO RUN: ###


* Navigate to the folder icon on the left
* Click on the upload file icon
* Upload agg.csv, conf.ini, testloss.txt, and best-model.keras from the github directory


In [ ]:
# Already install
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import matplotlib.pyplot as plt
import configparser
import seaborn as sns
from scikeras.wrappers import KerasRegressor

from sklearn.model_selection import RandomizedSearchCV

print(tf.__version__)

2.15.0


### TODOS: ###


*  Data preprocessing, visualize the relationship and choose the best ones (IN PROGRESS)
*  Show a graph that loss is reduced through training (DONE)
*  Add a metric for the model (DONE)








In [ ]:
def dropUnneeded(df, drop_non_correlated=False):
    try:
        df = df.drop("S&P OPEN", axis=1)
        df = df.drop("S&P HIGH", axis=1)
        df = df.drop("S&P LOW", axis=1)
    except:
        pass

    if drop_non_correlated:
        # List of non-correlated features to be dropped.
        try:
            non_correlated_features = ['1 Year T', '2 Year T', '3 Year T', '5 Year T', '7 Year T', '10 Year T', '30 Year T',
                                      'Mortgage 15 Yr',
                                      #  'Mortgage 30 Yr',
                                      'FFR',
                                      'UNEMP',
                                      'PRIME RATE',
                                      'UMich Consumer Sentiment'
                                      ]
            '''
            # Threshold for dropping non-correlated features
            threshold = 0.1  # or some other value close to 0

            # Identify non-correlated features
            non_correlated_features = corr_matrix.index[abs(corr_matrix['S&P CLOSE']) < threshold].tolist()
            '''
            # Drop the non-correlated features from the dataframe.
            df = df.drop(non_correlated_features, axis=1)

        except:
            pass

     # Convert to numpy
    data = df.to_numpy()

    # Format the data to a 3D shape so the LSTM can use it.
    # The resulting shapes will be as follows:
    #    dataX: (number_of_input_series, number_of_timesteps, number_of_features)
    #    dataY: (number_of_input_series, 1)
    dataX = []
    for i in range(n_past, len(data) - n_future + 1):
      dataX.append(data[i - int(conf['data']['n_past']):i, 0:data.shape[1]])

    return (df, np.array(dataX))


In [ ]:
# Returns a tuple of (df, dataX, dataY)
# Where df is the original dataframe, dataX is the formatted normalized input data,
# and dataY is the formatted normalized output data.
# The dataframe will not include the date column if df_dates is False.
def preprocess(n_past, n_future, label_column, null, df_dates=True, include_month_change=False, remove_label_from_input=False):
    # Load the data.
    # We'll forward fill any missing values, so long as there are no more than 5 of them consecutively,
    # then fill any remaining missing values with our null constant.
    df = pd.read_csv("./agg.csv").ffill(limit=5).fillna(null)

    if (not include_month_change and 'S&P MONTH CHANGE' in df.columns):
        df = df.drop('S&P MONTH CHANGE', axis=1)

    # Remove date column temporarily; not useful for normalizing
    ddf = df.iloc[:, 1:]

    # Mean normalize (standardization)
    mean = ddf.mean()
    std = ddf.std()
    ddf = (ddf - mean) / std

    # Convert to numpy
    data = ddf.to_numpy()

    # Format the data to a 3D shape so the LSTM can use it.
    # The resulting shapes will be as follows:
    #    dataX: (number_of_input_series, number_of_timesteps, number_of_features)
    #    dataY: (number_of_input_series, 1)
    dataX = []
    dataY = []

    for i in range(n_past, len(data) - n_future + 1):
        dataX.append(data[i - n_past:i, 0:data.shape[1]])
        dataY.append(data[i + n_future - 1:i + n_future, label_column])

    # Remove date column permanently if requested
    if not df_dates:
        df = df.iloc[:, 1:]

    # Delete S&P change, price open and price close columns from input data.
    if remove_label_from_input:
        dataX = np.delete(dataX, label_column, axis=2)

    # Return a tuple containing (df, dataX, dataY)
    return (df,np.array(dataX),np.array(dataY))

## Config the model

In [ ]:
conf = configparser.ConfigParser()
conf.read("./conf.ini")

BATCH_SIZE = 2
LEARNING_RATE = 0.003#0.005
EPOCHS = 2
WEIGHT_DECAY = 0.002#0.002
ACTIVATION = "tanh"
L1_REG = 0.001;

In [ ]:
# The arbitrary null value for missing data.
null = int(conf['data']['null'])

# How many timesteps in the future we're predicting
n_future = int(conf['data']['n_future'])
# The size of each input series (in timesteps).
n_past = int(conf['data']['n_past'])
# The column index containing the output, AFTER removing the date column.
label_column = int(conf['data']['label_column'])
# Include the S&P percentage change (by about a month)
include_month_change = conf['data'].getboolean('include_month_change')
# Remove the label from the features
remove_label_from_input = conf['data'].getboolean('remove_label_from_input')

KeyError: 'data'

In [ ]:
# Load the data
df, dataX, dataY = preprocess(n_past, n_future, label_column, null, df_dates=False, include_month_change=include_month_change, remove_label_from_input=remove_label_from_input)

print(df.head())
print("Shape:", dataX.shape)

In [ ]:
corr_matrix = df.corr(method="kendall")
plt.figure(figsize=(20,20))
sns.heatmap(corr_matrix, linewidths=2, xticklabels=True, yticklabels=True)
plt.show()

In [ ]:
label_correlations = corr_matrix['S&P CLOSE'].sort_values(ascending=False)
# label_matrix = corr_matrix[['S&P CLOSE']]
plt.figure(figsize=(10, 10))
# sns.heatmap(label_matrix, linewidths=2, xticklabels=True, yticklabels=True)
sns.heatmap(label_correlations.to_frame(), annot=True, fmt='.2f', center=0)

We can see from the Kendall correlation matrix that a few of our columns have a low correlation with our label (S&P Close). These include but are not limited to: VIX levels, 15 year mortgage rates, consumer sentiment, and investor sentiment.

We used Kendall Correlation due to its reputation of being a good coefficient for nonlinear data. While it isn't perfect, it's a good starting point for feature selection.

In [ ]:
# Drop the weak features
# df, dataX = dropUnneeded(df, drop_non_correlated=True)

In [ ]:
mean = df.mean()
std = df.std()

In [ ]:
# Train test split
trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=0.05, shuffle=False)

# Shuffle only the training data
trainRandomIndices = np.random.permutation(len(trainX))
trainX = trainX[trainRandomIndices]
trainY = trainY[trainRandomIndices]

In [ ]:
SHAPE = (dataX.shape[1],dataX.shape[2])

def build_model(activation):

    inputs = keras.Input(shape=SHAPE, dtype="float32", name="inputs")
    x = layers.LSTM(25, return_sequences=True, activation=activation, kernel_regularizer=regularizers.L1(L1_REG))(inputs)
    # x = layers.LSTM(40, return_sequences=True, activation=ACTIVATION)(x)
    x = layers.LSTM(25, return_sequences=False, activation=activation)(x)

    outputs = layers.Dense(1, activation=activation, name="outputs")(x) # activation="tanh",

    model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss="mse", run_eagerly=False, optimizer=(tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)), metrics=[keras.metrics.RootMeanSquaredError(), keras.losses.MeanAbsoluteError()])

    return model


In [ ]:
# Define the model
model = build_model(activation=ACTIVATION)

In [ ]:
# visualize the model
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png',show_layer_activations=True, show_shapes=True, show_layer_names=True)

In [ ]:
# Train the model
history = model.fit(trainX, trainY, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True)

# from keras.callbacks import History

# model = KerasRegressor(build_fn=build_model, verbose=0, activation="tanh")
# # Define the grid of hyperparameters to search
# param_dist = {
#   'activation': ['tanh', 'sigmoid'],
#   'optimizer': ['adam'],
#   'batch_size': [2, 4, 1],
#   'epochs': [4, 3, 2]
# }

# history = History()
# # Create a RandomizedSearchCV object
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, verbose=2)
# random_search_results = random_search.fit(trainX, trainY, callbacks=[history])

# print("Best: %f using %s" % (random_search_results.best_score_, random_search_results.best_params_))

In [ ]:
# import pickle

# with open('random_search_cv_results.pkl', 'wb') as f:
#     pickle.dump(random_search_results, f)

In [ ]:
# with open('random_search_cv_results.pkl', 'rb') as f:
#     results = pickle.load(f)

In [ ]:
# result_df = pd.DataFrame(results.cv_results_)
# result_df = result_df.drop(["param_batch_size", "param_optimizer", "param_epochs", "param_batch_size", "param_activation",
#                             "split0_test_score", "split1_test_score", "split2_test_score", "mean_fit_time", "mean_score_time", "std_fit_time", "std_score_time"], axis=1)
# result_df

In [ ]:
# best_activation = random_search_results.best_params_['activation']
# best_optimizer = random_search_results.best_params_['optimizer']
# best_batch_size = random_search_results.best_params_['batch_size']
# best_epochs = random_search_results.best_params_['epochs']

# print(best_batch_size)
# best_model = build_model(best_activation)

# history = best_model.fit(trainX, trainY, epochs=5, batch_size=best_batch_size, shuffle=True)

In [ ]:
# plot the loss
plt.plot(history.history['loss'], label='Train Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

In [ ]:
# Plot training & validation RSME
plt.plot(history.history['root_mean_squared_error'], label='Train RMSE')
plt.title('Model RMSE')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

In [ ]:
# plot mae
plt.plot(history.history['mean_absolute_error'], label='Train MAE')
plt.title('Model MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

In [ ]:
# Evaluate the model on the test data
train_loss = model.evaluate(trainX, trainY, verbose=0)
test_loss = model.evaluate(testX, testY, verbose=0)

print('Train Loss: ', train_loss)
print('Test Loss: ', test_loss)

## Compare current Loss and Model performance with previously trained model to update bessloss.txt file and best-model.keras

In [ ]:
bestlossr = open('bestloss.txt', 'r')
bestloss = bestlossr.read()

# Save the model if this is a new best loss ([0] = MSE)
if (not bestloss.replace(".", "").isnumeric()) or test_loss[0] < float(bestloss):
	print("New best loss! Saving model...")
	model.save("./best-model.keras")

	bestlossw = open('bestloss.txt', 'w')
	bestlossw.write(str(test_loss[0]))
	bestlossw.close()

	meanstd = open('meanstd.txt', 'w')
	meanstd.write(str(mean.to_list()) + "\n" + str(std.to_list()))
	meanstd.close()

else: print("Model not saved, best lost still", bestloss)

bestlossr.close()

In [ ]:
# Plot the predicted graph movement
testYReg = testY * std[df.columns[label_column]] + mean[df.columns[label_column]]
testYHat = model.predict(testX) * std[df.columns[label_column]] + mean[df.columns[label_column]]

print(testYReg.shape)
print(testYHat.shape)

for i in range(0, len(testYHat)):
	print("Predicted: ", testYHat[i], "Actual: ", testYReg[i])

print("Train loss:", train_loss)
print("Test loss:", test_loss)

plt.plot(testYReg, c="blue", label="Actual")
plt.plot(testYHat, c="red", label="Predicted")

plt.ylabel("S&P Index Level")
plt.legend()
plt.show()

In [ ]:
# Metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math

def mean_absolute_percentage_error(y_true, y_pred):
    # Convert inputs to numpy arrays for vectorized operation
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    # Avoid division by zero and only use non-zero elements for calculation
    mask = y_true != 0
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mape

mse = mean_squared_error(testYReg, testYHat)
mape = mean_absolute_percentage_error(testYReg, testYHat)
print("Root Mean Squared Error (MSE):", math.sqrt(mse))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))

In [ ]:
with open('log.csv', 'a') as logfile:
  # trainXLength, trainXTimesteps, trainXFeatures, Batch Size, Learning Rate, Epochs, Weight Decay, L1 Reg, Activation, layers, train MSE, test MSE, test RMSE, test MAE, test MAPE
  logfile.write(f"{trainX.shape[0]},{trainX.shape[1]},{trainX.shape[2]},{BATCH_SIZE},{LEARNING_RATE},{EPOCHS},{WEIGHT_DECAY},{L1_REG},{ACTIVATION},{len(model.layers)},")
  logfile.write(f"{train_loss[0]},{test_loss[0]},{test_loss[1]},{test_loss[2]},{mape}\n")